# **HQC Educational Implementation**

Hi! In this notebook, I’m going to explain step by step a tiny, educational version of the **HQC cryptosystem**. 

 **Important:** This is **not secure**, it’s only to help understand how the real HQC works. We use **tiny toy parameters** and a **simple error-correcting code (Hamming(7,4))** so you can see everything clearly.

We will follow this structure:

1. Imports and helper functions  
2. Hamming(7,4) code explanation and implementation  
3. HQC parameters  
4. HQC key generation, encryption, and decryption  
5. Demo with a simple 4-bit message  



In [1]:

# Imports
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple
import numpy as np



### Explanation:

1. `from __future__ import annotations` → Makes Python handle type hints nicely.
2. `from dataclasses import dataclass` → We'll use this to easily create classes that just hold data (like parameters of HQC).
3. `from typing import Tuple` → Lets us define that a function returns a tuple of multiple items.
4. `import numpy as np` → Numpy is a library that helps us work with arrays and vectors easily. All our vectors and codes are arrays of 0s and 1s.


In [2]:

# Helper Functions

def sample_sparse_binary_vector(n: int, weight: int, rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Create a binary vector of length n with exactly `weight` ones.
    """
    if weight < 0 or weight > n:
        raise ValueError("weight must satisfy 0 <= weight <= n")
    rng = rng or np.random.default_rng()
    vec = np.zeros(n, dtype=int)  # start with all zeros
    if weight == 0:
        return vec
    positions = rng.choice(n, size=weight, replace=False)  # choose random positions for 1s
    vec[positions] = 1
    return vec

def cyclic_convolution(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Multiply two binary vectors as polynomials modulo x^n - 1.
    This is like rotating and adding the products in GF(2).
    """
    n = a.shape[0]
    lin = np.convolve(a.astype(int), b.astype(int)) % 2
    out = np.zeros(n, dtype=int)
    out[:n] ^= lin[:n]
    for i in range(n, lin.shape[0]):
        out[i-n] ^= lin[i]  # wrap-around
    return out % 2


### Explanation of Helper Functions:

- `sample_sparse_binary_vector(n, weight)` → Creates a vector of length `n` filled with mostly 0s and exactly `weight` number of 1s.  
  - Example: n=7, weight=2 → `[0,1,0,0,1,0,0]`  
  - This is used to create **secret keys** and **errors** in HQC.  

- `cyclic_convolution(a, b)` → "Mixes" two binary vectors like HQC does in its math.

- First, think of each vector as a polynomial.  
  For example, vector `[1,0,1]` is like the polynomial 1 + 0*x + 1*x^2 → 1 + x^2.  

- GF(2) means **binary arithmetic**: we only use 0 and 1, and add/multiply modulo 2.  
  So 1 + 1 = 0, 1 * 1 = 1, 1 + 0 = 1, etc.  

- Cyclic convolution = multiply the two polynomials **and wrap around**:  
  - If the polynomial is longer than n terms, the extra terms "wrap" to the start.  
  - This keeps the result the same length as the original vectors.  

- Why we do this in HQC:  
  - HQC works in a special "ring" (think of it as a circle of numbers).  
  - This operation mixes the vectors but in a way that decryption can reverse it.  

- Bottom line for beginners: it's just a **reversible mixing** of two binary sequences.  



In [3]:

# Hamming(7,4) Code

class Hamming74:
    """
    Minimal Hamming(7,4) encoder/decoder for learning.
    - n=7 code length
    - k=4 message bits
    - Corrects 1 bit error
    """
    # Parity-check matrix
    H = np.array([
        [1,0,1,0,1,0,1],
        [0,1,1,0,0,1,1],
        [0,0,0,1,1,1,1]
    ], dtype=int)

    # Syndrome table: maps 3-bit syndrome -> error position (1..7), 0 = no error
    SYNDROME_TO_POS = {
        (0,0,0): 0,
        (1,0,0): 1,
        (0,1,0): 2,
        (1,1,0): 3,
        (0,0,1): 4,
        (1,0,1): 5,
        (0,1,1): 6,
        (1,1,1): 7,
    }

    def encode(d: np.ndarray) -> np.ndarray:
        """Encode 4-bit message into 7-bit codeword"""
        d1,d2,d3,d4 = [int(x)&1 for x in d]
        p1 = (d1 ^ d2 ^ d4) & 1
        p2 = (d1 ^ d3 ^ d4) & 1
        p3 = (d2 ^ d3 ^ d4) & 1
        return np.array([p1,p2,d1,p3,d2,d3,d4], dtype=int)

   
    def decode(r: np.ndarray) -> Tuple[np.ndarray,np.ndarray]:
        """Decode 7-bit vector, correct 1-bit error if exists"""
        s = (Hamming74.H @ r) % 2
        pos = Hamming74.SYNDROME_TO_POS.get(tuple(s),0)
        c_hat = r.copy()
        if pos != 0:
            c_hat[pos-1] ^= 1  # flip error
        m_hat = c_hat[[2,4,5,6]]  # extract original message
        return m_hat.astype(int), c_hat.astype(int)


### Explanation of Hamming(7,4) (with example)

- **Purpose:** Hamming(7,4) is a tiny **error-correcting code**.  
  - It takes **4 data bits** (original message) and turns them into **7 bits** by adding 3 **parity bits**.  
  - Parity bits help detect and correct a **single-bit error**.

- **Encoding Example:**  
  - Original message: `[1, 0, 1, 1]` (4 bits)  
  - `encode()` adds parity bits → `[0, 1, 1, 0, 0, 1, 1]` (7 bits)  

- **How decoding works:**  
  1. Suppose during transmission one bit flips by mistake: `[0, 1, 0, 0, 0, 1, 1]`  
  2. `decode()` calculates **syndrome** using the `H` matrix.  
     - Syndrome is a 3-bit vector that indicates **where the error is**.  
     - Example: syndrome = `(0, 1, 0)` → tells error is in **bit 2**.  
  3. Use `SYNDROME_TO_POS` → flip the wrong bit → `[0, 1, 1, 0, 0, 1, 1]`  
  4. Recover **original message** `[1, 0, 1, 1]` from corrected codeword.  

- **Bottom line:**  
  - `H` matrix + `SYNDROME_TO_POS` = **how we detect and fix one-bit errors**.  
  - For HQC, this is a **toy example** showing how a real error-correcting code works in the algorithm.


In [4]:

# HQC Parameters
rng = np.random.default_rng(12345)  # fixed seed for reproducibility in the demo
n=7
w=1 
wr1=1
wr2=0
we1=0 
we2=0

### Explanation of HQC Parameters

In HQC, all the numbers we use (keys, errors, etc.) are **binary vectors** of length `n`.  
Here `n = 7` because we are using a tiny demo version with Hamming(7,4) code.  

- `w` → This is the **weight of the secret key vectors** `x` and `y`.  
  - Weight means how many **1s** are in the vector(we called it its Humming weight).  
  - `w = 1` means each secret key vector has **exactly 1 “1”** and 6 zeros.  
  - Secret keys are **sparse** (mostly zeros) so encryption is easier to handle.

- `wr1` → Weight of `r1`, a random vector used in encryption.  
  - Only 1 bit is 1, the rest are 0.  
  - Helps mix the message without creating too many errors.

- `wr2` → Weight of `r2`, another random vector in encryption.  
  - Set to 0 in this demo so **the ciphertext doesn't get too noisy**.  

- `we1` and `we2` → These are **error vectors** added to the ciphertext.  
  - In real HQC, errors are used to make the scheme secure.  
  - Here, we set them to 0 to ensure **Hamming(7,4) can decode correctly**.

**Key idea for beginners:**  
- Think of these parameters as **how many 1s to put in each binary vector**.  
- They control **how “loud” the randomness and errors are** in encryption.  
- In a real HQC implementation, these numbers are much bigger, and the error-correcting code can fix more mistakes.


In [5]:
def keygen(n,w,rng):
        """Generate public and secret keys"""
        x = sample_sparse_binary_vector(n, w,rng)
        y = sample_sparse_binary_vector(n, w,rng)
        h = rng.integers(0,2,size=n)
        s = (x ^ cyclic_convolution(h,y)) % 2
        pk = (h,s)
        sk = (x,y)
        return pk, sk
pk, sk = keygen(n,w,rng)
print("Public key (h, s):\n h =", pk[0], "\n s =", pk[1])
print("Secret key (x, y):\n x =", sk[0], "\n y =", sk[1])





Public key (h, s):
 h = [1 0 0 1 1 1 1] 
 s = [1 1 0 0 0 1 1]
Secret key (x, y):
 x = [0 0 0 0 1 0 0] 
 y = [0 1 0 0 0 0 0]


### Explanation of `keygen()`

The `keygen()` function **creates the keys** for HQC encryption and decryption.

1. `x` and `y` → **secret keys**  
   - Each is a sparse binary vector of length `n` (mostly 0s, a few 1s).  
   - Example: `[0,0,0,0,1,0,0]` → only one 1, rest 0s.

2. `h` → **random public vector** of length `n`  
   - Example: `[1,0,0,1,1,1,1]`  

3. `s = x + h*y` (mod 2) → **part of the public key**  
   - Multiplying `h` by `y` using `cyclic_convolution`, then adding `x`  
   - This operation **mixes the secret keys** into the public key.  
   - Example:  
     ```
     x = [0,0,0,0,1,0,0]
     h = [1,0,0,1,1,1,1]
     y = [0,1,0,0,0,0,0]
     h*y = [0,1,0,0,0,0,0]  # via cyclic convolution
     s = x + h*y = [0,1,0,0,1,0,0]
     ```

4. `pk` → **public key** `(h, s)`  
5. `sk` → **secret key** `(x, y)`

**Key idea for beginners:**  
- Public key is **safe to share**, secret key is **kept private**.  
- The operation `s = x + h*y` ensures **the secret keys are hidden** in the public key.  
- In real HQC, vectors are much bigger, but the logic is the same.



In [6]:

# Encryption
  
def encrypt(m4, pk, rng, n, wr1, wr2, we1, we2):
        """Encrypt a 4-bit message using pk and Hamming(7,4) as C.

        Args:
            m4: np.ndarray shape (4,) with bits (the "data" part for Hamming(7,4)).

        Returns:
            (u, v): ciphertext polynomials in R (each np.ndarray of shape (n,)).
        """
        if pk is None:
            raise ValueError("Public key not initialized. Call keygen() first.")
        h, s = pk
        # Sample per-branch sparsities to keep decoder's error budget ≤ 1 for Hamming(7,4)
        r1 = sample_sparse_binary_vector(n,wr1,rng)
        r2 = sample_sparse_binary_vector(n,wr2,rng)
        e1 = sample_sparse_binary_vector(n,we1,rng)
        e2 = sample_sparse_binary_vector(n,we2,rng)
        # C.encode(m)
        c = Hamming74.encode(m4)  # length 7 = n
        # u = r1 + h · r2 + e1
        hr2 = cyclic_convolution(h, r2)
        u = (r1 ^ hr2 ^ e1) % 2
        # v = s · r2 + e2 + c
        sr2 = cyclic_convolution(s, r2)
        v = (sr2 ^ e2 ^ c) % 2
        return u, v


### Explanation of Encryption (`encrypt()`)

The `encrypt()` function **turns a 4-bit message into a ciphertext** using the public key `(h, s)`.

1. **Input:**  
   - `m4` → the 4-bit message to encrypt, e.g., `[1,0,1,1]`

2. **Step 1: Sample random vectors**  
- These vectors **hide the message** and make it hard for an attacker to guess the original message  
- We keep them small so the Hamming code can still decode correctly

3. **Step 2: Encode the message with Hamming(7,4)**  
- Turns the 4-bit message into a 7-bit codeword  
- Adds **redundancy** so errors can be corrected later

4. **Step 3: First part of ciphertext (`u`)**

- `h * r2` → think of it as **mixing two sequences of 0s and 1s together** in a special way (like shuffling them).  
- Then we **combine** this with `r1` and `e1`:
  - Instead of "XOR", just imagine **adding 0s and 1s with a twist**:
    - 0 + 0 = 0 → stays 0  
    - 1 + 0 = 1 → stays 1  
    - 1 + 1 = 0 → flips back to 0  
- The result is `u`, a scrambled version of random vectors and part of the public key.

5.**Step 4: Second part of ciphertext (`v`)**

- `s * r2` → again, **mixing two sequences** like before.  
- Then we **add the error vector** `e2` and the encoded message `c`:  
  - Think of it as **layering**: first the mix, then sprinkle a little noise (`e2`), then put the actual message on top (`c`).  
- The result is `v`, the second part of the ciphertext.

**Bottom line for beginners:**

- `u` and `v` are just two **scrambled sequences of 0s and 1s**.  
- The way we scramble ensures only someone with the **secret key** can **unscramble** and recover the original message.  
- You don’t need to understand XOR or polynomials fully, just remember: **it mixes and hides the message in a reversible way**.


In [7]:

# Decryption

def decrypt(ct,sk):
        if sk is None:
            raise ValueError("Call keygen() first.")
        x,y = sk
        u,v = ct
        t = (v ^ cyclic_convolution(u,y)) % 2  # v - u*y
        m_hat, c_hat = Hamming74.decode(t)
        return m_hat, t, c_hat


### Explanation of Decryption 
- **Step 1: Remove secret key effect**
  - `t = v - u*y` → think of it as **undoing the scramble** we did during encryption.  
  - It's like: "I know the secret key, so I can reverse the layers and get something close to the original message."

- **Step 2: Error correction**
  - `Hamming74.decode(t)` → the tiny Hamming code looks at `t` and **fixes 1-bit mistakes** (if any).  
  - This ensures that small errors added during encryption or transmission don’t break the message.

- **Step 3: Outputs**
  1. `m_hat` → the **4-bit original message** recovered.  
  2. `t` → the scrambled-but-key-removed vector that goes into the decoder.  
  3. `c_hat` → the **corrected 7-bit codeword**, showing what the decoder actually corrected.
 
Decryption is just **unscrambling using the secret key** and **letting Hamming code fix tiny errors** to get back the original message safely.



In [8]:
# Demo

# keep error budget small for Hamming(7,4)
pk, sk = keygen(n,w,rng)
print("Public key (h, s):\n h =", pk[0], "\n s =", pk[1])
print("Secret key (x, y):\n x =", sk[0], "\n y =", sk[1])

# 4-bit message, e.g., 1011
m = np.array([1, 0, 1, 1], dtype=int)
print("\nPlain message m (4 bits):", m)

u,v= encrypt(m, pk, rng, n, wr1, wr2, we1, we2=we2) 
print("\nCiphertext (u, v):\n u =", u, "\n v =", v)

m_hat, t, c_hat = decrypt((u, v),sk)
print("\nDecryption intermediate t = v + u·y:", t)
print("Corrected codeword from decoder:", c_hat)
print("Recovered message m_hat:", m_hat)

ok = np.array_equal(m, m_hat)
print("\nSUCCESS:", ok)



Public key (h, s):
 h = [0 1 1 0 0 0 1] 
 s = [1 0 1 0 1 0 1]
Secret key (x, y):
 x = [0 0 1 0 0 0 0] 
 y = [0 0 0 0 0 1 0]

Plain message m (4 bits): [1 0 1 1]

Ciphertext (u, v):
 u = [0 0 0 0 1 0 0] 
 v = [0 1 1 0 0 1 1]

Decryption intermediate t = v + u·y: [0 1 0 0 0 1 1]
Corrected codeword from decoder: [0 1 1 0 0 1 1]
Recovered message m_hat: [1 0 1 1]

SUCCESS: True


### Final Notes

- This **educational HQC** mirrors the real HQC flow: **KeyGen → Encrypt → Decrypt**.  
- **We use Hamming(7,4)** so we can see **every bit** and understand how the error-correcting step works.  
- In real HQC, parameters are **much larger** and use BCH codes instead of Hamming.  
- Try running the notebook, changing the 4-bit message, and watch how the intermediate vector `t` and the corrected codeword `c_hat` behave.  
- You can also experiment with different secret keys or random vectors to see the effect on the ciphertext.  
- This notebook is meant for learning and visualization; it is **not secure for real encryption**.

